In [331]:
import plotly.graph_objects as go
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from IPython.display import display


# **Data**

In [332]:
ticker = "BTC-USD"

data = yf.download(ticker, start="2018-01-01", end="2024-12-27", auto_adjust=True)

data.columns = [col[0] if col[1] == '' else col[0] for col in data.columns]


columns_to_keep = ['Open', 'High', 'Low', 'Close', 'Volume']
data = data[columns_to_keep]

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,10291200000
2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,16846600192
2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,16871900160
2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,21783199744
2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,23840899072
...,...,...,...,...,...
2024-12-22,97218.320312,97360.265625,94202.187500,95104.937500,43147981314
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,65239002919
2024-12-24,94684.343750,99404.062500,93448.015625,98676.093750,47114953674


# **Strategies**

In [333]:
class HybridATRCCIStrategy(bt.Strategy):
    params = (
        ('atr_period', 14),  # ATR periyodu
        ('cci_period', 20),  # CCI periyodu
        ('cci_threshold', 100),  # CCI eşik değeri
        ('period', 20)
    )

    def __init__(self):
        self.buy_price = None
        self.sell_price = None

        self.buy_signals = []
        self.sell_signals = []

        # ATR Hesaplama
        self.atr = bt.indicators.ATR(self.data, period=self.params.atr_period)

        # CCI Hesaplama
        self.cci = bt.indicators.CommodityChannelIndex(self.data, period=self.params.cci_period)
        self.sma = bt.indicators.WeightedMovingAverage(period=self.params.period)
        self.std = bt.indicators.StandardDeviation(self.data.close, period=self.params.period)

    def next(self):
        cash = self.broker.get_cash()  

        if self.cci[0] > self.params.cci_threshold and self.atr[0] > self.atr[-1] and not self.position:
            self.buy()
            self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        elif self.cci[0] < -self.params.cci_threshold and self.atr[0] < self.atr[-1] and self.position:
            self.sell(size=self.position.size)
            self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

**MACD RSI Strategy**

In [334]:

class MACD_RSI_Strategy(bt.Strategy):
    params = (
        ('macd_fast', 12),  # Fast EMA period
        ('macd_slow', 26),  # Slow EMA period
        ('macd_signal', 9),  # Signal line period
        ('rsi_period', 14),  # RSI period
        ('rsi_upper', 70),  # Overbought level
        ('rsi_lower', 30),  # Oversold level
    )

    def __init__(self):
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal,
        )
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)

        self.buy_signals = []
        self.sell_signals = []

    def next(self):

        if self.macd.macd[0] > self.macd.signal[0] and self.rsi[0] < self.params.rsi_upper:
            if not self.position:
                self.buy()
                self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        elif self.macd.macd[0] < self.macd.signal[0] and self.rsi[0] > self.params.rsi_lower:
            if self.position:
                self.sell()
                self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

In [335]:
class BollingerBand_MACD(bt.Strategy):

    params = (
        ('period', 20),            # Period for Bollinger Bands
        ('devfactor', 1.5),          # Standard deviation factor for Bollinger Bands
        ('macd_slow', 26),         # Slow period for MACD
        ('macd_fast', 12),         # Fast period for MACD
        ('macd_signal', 9),        # Signal line period for MACD
    )

    def __init__(self):
        self.buy_signals = []
        self.sell_signals = []
        self.upper_bounds = []
        self.lower_bounds = []

        # Bollinger Bands
        self.boll = bt.indicators.BollingerBands(
            self.data.close,
            period=self.params.period,
            devfactor=self.params.devfactor
        )

        # MACD
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal,
        )

    def next(self):
        # Log upper and lower Bollinger Bands
        upper_bound = self.boll.lines.top[0]
        lower_bound = self.boll.lines.bot[0]

        self.upper_bounds.append((self.data.datetime.datetime(0), upper_bound))
        self.lower_bounds.append((self.data.datetime.datetime(0), lower_bound))

        # Entry Condition: Price below lower Bollinger Band and MACD bullish crossover
        if not self.position and self.data.close[0] < lower_bound and self.macd.macd[0] > self.macd.signal[0]:
            self.buy()
            self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        # Exit Condition: Price above upper Bollinger Band and MACD bearish crossover
        elif self.position and self.data.close[0] > upper_bound and self.macd.macd[0] < self.macd.signal[0]:
            self.sell()
            self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

In [336]:
class BollingerBand_ADX(bt.Strategy):

    params = (
        ('period', 20),            # Period for Bollinger Bands
        ('devfactor', 1.5),          # Standard deviation factor for Bollinger Bands
        ('adx_period', 14)

    )

    def __init__(self):
        self.buy_signals = []
        self.sell_signals = []
        self.upper_bounds = []
        self.lower_bounds = []

        # Bollinger Bands
        self.boll = bt.indicators.BollingerBands(
            self.data.close,
            period=self.params.period,
            devfactor=self.params.devfactor
        )

        self.adx = bt.indicators.ADX(self.data, period=self.params.adx_period)

    def next(self):
        # Log upper and lower Bollinger Bands
        upper_bound = self.boll.lines.top[0]
        lower_bound = self.boll.lines.bot[0]

        self.upper_bounds.append((self.data.datetime.datetime(0), upper_bound))
        self.lower_bounds.append((self.data.datetime.datetime(0), lower_bound))

        if self.adx[0] < 25:  # Weak trend
            return
        
        # Entry Condition: Price below lower Bollinger Band and MACD bullish crossover
        if not self.position and self.data.close[0] < lower_bound:
            self.buy()
            self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        # Exit Condition: Price above upper Bollinger Band and MACD bearish crossover
        elif self.position and self.data.close[0] > upper_bound:
            self.sell()
            self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

In [337]:

class Bollinger_RSI_Strategy(bt.Strategy):
    params = (
        ('boll_period', 20),  # Bollinger Bands period
        ('boll_dev', 2),      # Standard deviation for Bollinger Bands
        ('rsi_period', 14),  # RSI period
        ('rsi_upper', 70),   # Overbought level
        ('rsi_lower', 30),   # Oversold level
    )

    def __init__(self):
        self.boll = bt.indicators.BollingerBands(
            self.data.close,
            period=self.params.boll_period,
            devfactor=self.params.boll_dev
        )
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)

        self.buy_signals = []
        self.sell_signals = []
        print("Bollinger_RSI_Strategy")

    def next(self):

        # Entry Condition: Price below lower Bollinger Band and RSI oversold
        if self.data.close[0] < self.boll.lines.bot[0] and self.rsi[0] < self.params.rsi_lower:
            if not self.position:
                self.buy()
                self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        # Exit Condition: Price above upper Bollinger Band and RSI overbought
        elif self.data.close[0] > self.boll.lines.top[0] and self.rsi[0] > self.params.rsi_upper:
            if self.position:
                self.sell()
                self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

class ADX_MACD_Strategy(bt.Strategy):
    params = (
        ('macd_fast', 12),  # Fast EMA period
        ('macd_slow', 26),  # Slow EMA period
        ('macd_signal', 9),  # Signal line period
        ('adx_period', 14),  # ADX period
        ('adx_threshold', 25),  # ADX threshold for strong trend
    )

    def __init__(self):
        print("ADX_MACD_Strategy")
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal,
        )
        self.adx = bt.indicators.ADX(period=self.params.adx_period)

        self.buy_signals = []
        self.sell_signals = []

    def next(self):

        # Entry Condition: MACD bullish crossover and strong ADX
        if self.macd.macd[0] > self.macd.signal[0] and self.adx[0] > self.params.adx_threshold:
            if not self.position:
                self.buy()
                self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        # Exit Condition: MACD bearish crossover and ADX remains strong
        elif self.macd.macd[0] < self.macd.signal[0] and self.adx[0] > self.params.adx_threshold:
            if self.position:
                self.sell()
                self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

class ATR_TrailingStop_Strategy(bt.Strategy):
    params = (
        ('atr_period', 14),  # ATR period
        ('atr_multiplier', 3),  # Multiplier for trailing stop
    )

    def __init__(self):
        self.atr = bt.indicators.ATR(self.data, period=self.params.atr_period)
        self.trailing_stop = None

        self.buy_signals = []
        self.sell_signals = []

        print("ATR_TrailingStop_Strategy")

    def next(self):
        if not self.position:
            # Entry Condition: A simple example (price above a certain level)
            if self.data.close[0] > self.data.open[0]:
                self.buy()
                self.trailing_stop = self.data.close[0] - (self.atr[0] * self.params.atr_multiplier)
                self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        else:
            # Exit Condition: Trailing stop
            self.trailing_stop = max(self.trailing_stop, self.data.close[0] - (self.atr[0] * self.params.atr_multiplier))

            if self.data.close[0] < self.trailing_stop:
                self.sell()
                self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

class Momentum_Strategy(bt.Strategy):
    params = (
        ('momentum_period', 14),  # Momentum period
        ('upper_threshold', 100),  # Overbought threshold
        ('lower_threshold', -100),  # Oversold threshold
    )

    def __init__(self):
        self.momentum = bt.indicators.Momentum(self.data.close, period=self.params.momentum_period)

        self.buy_signals = []
        self.sell_signals = []

        print("Momentum_Strategy")

    def next(self):

        # Entry Condition: Momentum crosses above lower threshold
        if self.momentum[0] > self.params.lower_threshold and self.momentum[-1] <= self.params.lower_threshold:
            if not self.position:
                self.buy()
                self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        # Exit Condition: Momentum crosses below upper threshold
        elif self.momentum[0] < self.params.upper_threshold and self.momentum[-1] >= self.params.upper_threshold:
            if self.position:
                self.sell()
                self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

class Stochastic_MACD_Strategy(bt.Strategy):
    params = (
        ('stoch_k_period', 14),  # Stochastic %K period
        ('stoch_d_period', 3),   # Stochastic %D period
        ('macd_fast', 12),       # MACD fast period
        ('macd_slow', 26),       # MACD slow period
        ('macd_signal', 9),      # MACD signal period
    )

    def __init__(self):
        # Initialize the Stochastic indicator
        self.stochastic = bt.indicators.Stochastic(
            self.data,  # Use the entire data object which includes high, low, close
            period=self.params.stoch_k_period, 
            period_dfast=self.params.stoch_d_period
        )

        # Initialize the MACD indicator
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal
        )

        # Lists to store signals for logging or plotting
        self.buy_signals = []
        self.sell_signals = []

        print("Stochastic_MACD_Strategy Initialized")

    def next(self):
        # Entry Condition: Stochastic %K crosses above %D and MACD bullish crossover
        if (self.stochastic.percK[0] > self.stochastic.percD[0] and
                self.stochastic.percK[-1] <= self.stochastic.percD[-1] and
                self.macd.macd[0] > self.macd.signal[0]):
            if not self.position:  # Only buy if no position is open
                self.buy()
                self.buy_signals.append((self.data.datetime.date(0), self.data.close[0]))
                print(f"Buy Signal: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit Condition: Stochastic %K crosses below %D and MACD bearish crossover
        elif (self.stochastic.percK[0] < self.stochastic.percD[0] and
              self.stochastic.percK[-1] >= self.stochastic.percD[-1] and
              self.macd.macd[0] < self.macd.signal[0]):
            if self.position:  # Only sell if a position is open
                self.sell()
                self.sell_signals.append((self.data.datetime.date(0), self.data.close[0]))
                print(f"Sell Signal: {self.data.datetime.date(0)} @ {self.data.close[0]}")



class SuperTrend(bt.Indicator):
    lines = ('supertrend', 'upperband', 'lowerband', 'trend')
    params = (
        ('period', 7),
        ('multiplier', 3),
    )

    def __init__(self):
        self.atr = bt.indicators.ATR(self.data, period=self.params.period)
        hl2 = (self.data.high + self.data.low) / 2

        # Calculate the upper and lower bands
        self.lines.upperband = hl2 + (self.params.multiplier * self.atr)
        self.lines.lowerband = hl2 - (self.params.multiplier * self.atr)

        # Trend direction line
        self.lines.trend = bt.LineNum(0)  # Initialize trend line with 0

    def next(self):
        # Calculate Supertrend and trend direction
        if self.data.close[0] > self.lines.upperband[-1]:
            self.lines.supertrend[0] = self.lines.lowerband[0]
            self.lines.trend[0] = 1  # Bullish trend
        elif self.data.close[0] < self.lines.lowerband[-1]:
            self.lines.supertrend[0] = self.lines.upperband[0]
            self.lines.trend[0] = -1  # Bearish trend
        else:
            self.lines.supertrend[0] = self.lines.supertrend[-1]
            self.lines.trend[0] = self.lines.trend[-1]

class Supertrend_MACD_Strategy(bt.Strategy):
    params = (
        ('supertrend_period', 7),
        ('supertrend_multiplier', 3),
        ('macd_fast', 12),
        ('macd_slow', 26),
        ('macd_signal', 9),
    )

    def __init__(self):
        # Initialize SuperTrend indicator
        self.supertrend = SuperTrend(
            self.data,
            period=self.params.supertrend_period,
            multiplier=self.params.supertrend_multiplier
        )

        # Initialize MACD indicator
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal
        )

    def next(self):
        # Entry: Supertrend bullish and MACD bullish crossover
        if self.supertrend.trend[0] == 1 and self.macd.macd[0] > self.macd.signal[0]:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit: Supertrend bearish or MACD bearish crossover
        elif self.supertrend.trend[0] == -1 or self.macd.macd[0] < self.macd.signal[0]:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")

class HeikinAshiStrategy(bt.Strategy):
    def __init__(self):
        self.ha_open = bt.indicators.HeikinAshi(self.data).lines.open
        self.ha_close = bt.indicators.HeikinAshi(self.data).lines.close

    def next(self):
        # Entry: Heikin-Ashi close > Heikin-Ashi open (bullish candle)
        if self.ha_close[0] > self.ha_open[0]:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit: Heikin-Ashi close < Heikin-Ashi open (bearish candle)
        elif self.ha_close[0] < self.ha_open[0]:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")


class VWMA_RSI_Strategy(bt.Strategy):
    params = (
        ('vwma_period', 20),
        ('rsi_period', 14),
        ('rsi_upper', 70),
        ('rsi_lower', 30),
    )

    def __init__(self):
        self.vwma = bt.indicators.WeightedMovingAverage(self.data.close, period=self.params.vwma_period)
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)

    def next(self):
        # Entry: Price above VWMA and RSI oversold
        if self.data.close[0] > self.vwma[0] and self.rsi[0] < self.params.rsi_lower:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit: Price below VWMA and RSI overbought
        elif self.data.close[0] < self.vwma[0] and self.rsi[0] > self.params.rsi_upper:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")


class DonchianChannel(bt.Indicator):
    lines = ('top', 'bot', 'mid')  # Top, Bottom, and Middle lines
    params = (
        ('period', 20),  # Lookback period
    )

    def __init__(self):
        self.lines.top = bt.indicators.Highest(self.data.high, period=self.params.period)
        self.lines.bot = bt.indicators.Lowest(self.data.low, period=self.params.period)
        self.lines.mid = (self.lines.top + self.lines.bot) / 2  # Middle line

class Donchian_ADX_Strategy(bt.Strategy):
    params = (
        ('donchian_period', 20),  # Lookback period for Donchian Channels
        ('adx_period', 14),       # Lookback period for ADX
        ('adx_threshold', 25),    # Threshold for ADX to confirm a strong trend
    )

    def __init__(self):
        # Initialize Donchian Channels
        self.donchian = DonchianChannel(self.data, period=self.params.donchian_period)

        # Initialize ADX
        self.adx = bt.indicators.ADX(self.data, period=self.params.adx_period)

    def next(self):
        # Ensure all indicators have enough data
        if len(self.data) < max(self.params.donchian_period, self.params.adx_period):
            return

        # Entry Condition: Breakout above Donchian upper band and strong ADX
        if self.data.close[0] > self.donchian.lines.top[0] and self.adx[0] > self.params.adx_threshold:
            if not self.position:  # Only buy if not already in a position
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit Condition: Price drops below the Donchian lower band
        elif self.data.close[0] < self.donchian.lines.bot[0]:
            if self.position:  # Only sell if in a position
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")

class IchimokuBreakoutStrategy(bt.Strategy):
    def __init__(self):
        self.ichimoku = bt.indicators.Ichimoku()

    def next(self):
        # Entry: Price breaks above the cloud
        if self.data.close[0] > self.ichimoku.senkou_span_a[0] and self.data.close[0] > self.ichimoku.senkou_span_b[0]:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit: Price breaks below the cloud
        elif self.data.close[0] < self.ichimoku.senkou_span_a[0] and self.data.close[0] < self.ichimoku.senkou_span_b[0]:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")


In [338]:
class BuyAndHold(bt.Strategy):
    def __init__(self):
        self.buy_signals = []
        self.sell_signals = []

    def nextstart(self):
        self.order = self.buy()
        self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))


class Bollinger_MACD_ATR_Strategy(bt.Strategy):
    params = (
        ('boll_period', 20),        # Bollinger Bands period
        ('boll_dev', 2),            # Bollinger Bands deviation factor
        ('macd_fast', 12),          # MACD fast period
        ('macd_slow', 26),          # MACD slow period
        ('macd_signal', 9),         # MACD signal period
        ('atr_period', 14),         # ATR period
        ('atr_multiplier', 3),      # ATR trailing stop multiplier
        ('rsi_period', 14),         # RSI period
        ('rsi_overbought', 70),     # RSI overbought level
        ('rsi_oversold', 30),       # RSI oversold level
    )

    def __init__(self):
        # Bollinger Bands
        self.boll = bt.indicators.BollingerBands(
            self.data.close,
            period=self.params.boll_period,
            devfactor=self.params.boll_dev
        )

        # MACD
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal
        )

        # ATR Trailing Stop
        self.atr = bt.indicators.ATR(self.data, period=self.params.atr_period)
        self.trailing_stop = None

        # RSI for additional entry filter
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)

        # Logging buy/sell signals
        self.buy_signals = []
        self.sell_signals = []

    def next(self):
        # Entry Condition: Price below lower Bollinger Band and MACD bullish crossover
        if (self.data.close[0] < self.boll.lines.bot[0] and 
            self.macd.macd[0] > self.macd.signal[0] and
            self.rsi[0] < self.params.rsi_oversold):
            
            if not self.position:
                self.buy()
                self.trailing_stop = self.data.close[0] - (self.atr[0] * self.params.atr_multiplier)
                self.buy_signals.append((self.data.datetime.date(0), self.data.close[0]))
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit Condition: Price above upper Bollinger Band and MACD bearish crossover
        elif (self.data.close[0] > self.boll.lines.top[0] and 
              self.macd.macd[0] < self.macd.signal[0] and
              self.rsi[0] > self.params.rsi_overbought):
            
            if self.position:
                self.sell()
                self.sell_signals.append((self.data.datetime.date(0), self.data.close[0]))
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Trailing Stop Update
        if self.position:
            self.trailing_stop = max(self.trailing_stop, self.data.close[0] - (self.atr[0] * self.params.atr_multiplier))
            if self.data.close[0] < self.trailing_stop:
                self.sell()
                self.sell_signals.append((self.data.datetime.date(0), self.data.close[0]))
                print(f"TRAILING STOP SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")

class TripleMovingAverageStrategy(bt.Strategy):
    params = (
        ('fast_period', 10),
        ('medium_period', 20),
        ('slow_period', 50),
    )

    def __init__(self):
        self.fast_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast_period)
        self.medium_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.medium_period)
        self.slow_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow_period)

    def next(self):
        # Entry condition
        if self.fast_ma[0] > self.medium_ma[0] > self.slow_ma[0]:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit condition
        elif self.fast_ma[0] < self.medium_ma[0] < self.slow_ma[0]:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")

class TripleMA_MACD_Strategy(bt.Strategy):
    params = (
        ('ma_short', 10),
        ('ma_medium', 30),
        ('ma_long', 50),
        ('macd_fast', 12),
        ('macd_slow', 26),
        ('macd_signal', 9),
    )

    def __init__(self):
        self.ma_short = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.ma_short)
        self.ma_medium = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.ma_medium)
        self.ma_long = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.ma_long)
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal
        )

    def next(self):
        if self.ma_short[0] > self.ma_medium[0] > self.ma_long[0] and self.macd.macd[0] > self.macd.signal[0]:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")
        elif self.ma_short[0] < self.ma_medium[0] < self.ma_long[0] or self.macd.macd[0] < self.macd.signal[0]:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")

class PSAR_ADX_Strategy(bt.Strategy):
    params = (
        ('adx_period', 14),
        ('adx_threshold', 25),
        ('psar_acceleration', 0.02),
        ('psar_max', 0.2),
    )

    def __init__(self):
        self.psar = bt.indicators.ParabolicSAR(
            self.data,
            af=self.params.psar_acceleration,
            afmax=self.params.psar_max
        )
        self.adx = bt.indicators.ADX(self.data, period=self.params.adx_period)

    def next(self):
        if self.data.close[0] > self.psar[0] and self.adx[0] > self.params.adx_threshold:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")
        elif self.data.close[0] < self.psar[0]:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")

class VWAP_Bollinger_Stochastic(bt.Strategy):
    params = (
        ('boll_period', 20),
        ('boll_dev', 2),
        ('stoch_k', 14),
        ('stoch_d', 3),
    )

    def __init__(self):
        self.vwap_boll = bt.indicators.BollingerBands(
            self.data.close,
            period=self.params.boll_period,
            devfactor=self.params.boll_dev
        )
        self.stoch = bt.indicators.Stochastic(
            self.data,
            period=self.params.stoch_k,
            period_dfast=self.params.stoch_d
        )

    def next(self):
        if self.data.close[0] > self.vwap_boll.lines.bot[0] and self.stoch.percK[0] > self.stoch.percD[0]:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")
        elif self.data.close[0] < self.vwap_boll.lines.top[0] and self.stoch.percK[0] < self.stoch.percD[0]:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")

class KeltnerChannel(bt.Indicator):
    lines = ('top', 'mid', 'bot')  # Upper band, middle line, lower band
    params = (
        ('period', 20),
        ('atr_multiplier', 2),
    )

    def __init__(self):
        self.atr = bt.indicators.ATR(self.data, period=self.params.period)
        self.lines.mid = bt.indicators.EMA(self.data.close, period=self.params.period)
        self.lines.top = self.lines.mid + (self.atr * self.params.atr_multiplier)
        self.lines.bot = self.lines.mid - (self.atr * self.params.atr_multiplier)


class Keltner_Momentum_Strategy(bt.Strategy):
    params = (
        ('keltner_period', 20),
        ('atr_multiplier', 2),
        ('momentum_period', 14),
    )

    def __init__(self):
        # Initialize Keltner Channel
        self.keltner = KeltnerChannel(
            self.data,
            period=self.params.keltner_period,
            atr_multiplier=self.params.atr_multiplier
        )
        # Initialize Momentum Indicator
        self.momentum = bt.indicators.Momentum(self.data.close, period=self.params.momentum_period)

    def next(self):
        # Entry: Price above upper Keltner band and momentum is positive
        if self.data.close[0] > self.keltner.lines.top[0] and self.momentum[0] > 0:
            if not self.position:
                self.buy()
                print(f"BUY: {self.data.datetime.date(0)} @ {self.data.close[0]}")

        # Exit: Price below lower Keltner band and momentum is negative
        elif self.data.close[0] < self.keltner.lines.bot[0] and self.momentum[0] < 0:
            if self.position:
                self.sell()
                print(f"SELL: {self.data.datetime.date(0)} @ {self.data.close[0]}")


# **BackTrader**

In [339]:
strategies = [BollingerBand_MACD, MACD_RSI_Strategy, BuyAndHold, HybridATRCCIStrategy, Bollinger_RSI_Strategy, BollingerBand_ADX, Stochastic_MACD_Strategy, Momentum_Strategy,
              ATR_TrailingStop_Strategy, ADX_MACD_Strategy, Bollinger_MACD_ATR_Strategy, Supertrend_MACD_Strategy, IchimokuBreakoutStrategy, Donchian_ADX_Strategy, VWMA_RSI_Strategy,
              HeikinAshiStrategy, TripleMovingAverageStrategy, TripleMA_MACD_Strategy, PSAR_ADX_Strategy, VWAP_Bollinger_Stochastic,
              Keltner_Momentum_Strategy]

strategy_dict = {}

for strategy in strategies:
    
    cerebro = bt.Cerebro()

    cerebro.addstrategy(strategy)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=99)


    data_feed = bt.feeds.PandasData(dataname=data)

    cerebro.adddata(data_feed)

    cerebro.broker.setcash(100000)

    cerebro.broker.setcommission(commission=0.001)

    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    results = cerebro.run()

    strategy_dict[strategy.__name__] = {
        'cerebro': cerebro,
        'results': results
    }

Bollinger_RSI_Strategy
Stochastic_MACD_Strategy Initialized
Buy Signal: 2018-02-27 @ 10725.599609375
Sell Signal: 2018-03-14 @ 8269.8095703125
Buy Signal: 2018-04-08 @ 7023.52001953125
Sell Signal: 2018-05-16 @ 8368.830078125
Buy Signal: 2018-06-30 @ 6404.0
Sell Signal: 2018-08-12 @ 6322.68994140625
Buy Signal: 2018-08-21 @ 6488.759765625
Sell Signal: 2018-09-17 @ 6281.2001953125
Buy Signal: 2018-09-28 @ 6644.1298828125
Sell Signal: 2018-11-24 @ 3880.760009765625
Buy Signal: 2018-12-04 @ 3956.893798828125
Sell Signal: 2019-01-20 @ 3601.013671875
Buy Signal: 2019-02-08 @ 3666.7802734375
Sell Signal: 2019-03-03 @ 3847.17578125
Buy Signal: 2019-04-02 @ 4879.8779296875
Sell Signal: 2019-04-25 @ 5210.515625
Buy Signal: 2019-05-08 @ 5982.45751953125
Sell Signal: 2019-06-03 @ 8208.9951171875
Buy Signal: 2019-06-19 @ 9273.521484375
Sell Signal: 2019-07-05 @ 10978.4599609375
Buy Signal: 2019-08-09 @ 11862.9365234375
Sell Signal: 2019-08-22 @ 10131.0556640625
Buy Signal: 2019-09-13 @ 10360.54687

In [340]:
# rtot: The total return for the entire backtest period (29.66%)
# ravg: The average per-bar return (e.g., if these are daily bars, this is the average daily return)
# rnorm: The normalized (annualized) return (23.00% per year, if daily bars)
# rnorm100: The same normalized return as a percentage (23.00%)

strategy_dict['BollingerBand_MACD']['results'][0].analyzers.returns.get_analysis()

OrderedDict([('rtot', 0.4758331962477702),
             ('ravg', 0.00018645501420367172),
             ('rnorm', 0.04810803097683325),
             ('rnorm100', 4.8108030976833245)])

In [341]:
# 1) The current drawdown length (in bars).
# 2) The current drawdown in percentage (i.e., ~6.11%).
# 3) The current drawdown in monetary terms (e.g., $8,753.39).
# 4) The length (in bars) of the maximum drawdown observed.
# 5) The maximum drawdown percentage (~20.83%).
# 6) The maximum drawdown in monetary terms (e.g., $27,025.29).

strategy_dict['BollingerBand_MACD']['results'][0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 413),
                 ('drawdown', 0.07468625801129163),
                 ('moneydown', 120.28650643370929),
                 ('max',
                  AutoOrderedDict([('len', 1115),
                                   ('drawdown', 70.64555137548084),
                                   ('moneydown', 87323.0825283324)]))])

In [342]:
strategy_dict['BuyAndHold']['results'][0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 9),
                 ('drawdown', 9.732234638307926),
                 ('moneydown', 74990.73698604933),
                 ('max',
                  AutoOrderedDict([('len', 1045),
                                   ('drawdown', 80.81101984707405),
                                   ('moneydown', 375345.9550476072)]))])

In [343]:
strategy_dict['BuyAndHold']['results'][0].analyzers.timereturn.get_analysis()

OrderedDict([(datetime.datetime(2018, 1, 1, 0, 0), 0.0),
             (datetime.datetime(2018, 1, 2, 0, 0), 0.0973874472263907),
             (datetime.datetime(2018, 1, 3, 0, 0), 0.014459726131796025),
             (datetime.datetime(2018, 1, 4, 0, 0), 0.02592866695065088),
             (datetime.datetime(2018, 1, 5, 0, 0), 0.1161672714565316),
             (datetime.datetime(2018, 1, 6, 0, 0), 0.005544174451358463),
             (datetime.datetime(2018, 1, 7, 0, 0), -0.05934338821039331),
             (datetime.datetime(2018, 1, 8, 0, 0), -0.07860346703544696),
             (datetime.datetime(2018, 1, 9, 0, 0), -0.037496785442737934),
             (datetime.datetime(2018, 1, 10, 0, 0), 0.02561695237661965),
             (datetime.datetime(2018, 1, 11, 0, 0), -0.1036033029344956),
             (datetime.datetime(2018, 1, 12, 0, 0), 0.042382094544157534),
             (datetime.datetime(2018, 1, 13, 0, 0), 0.02685132588451311),
             (datetime.datetime(2018, 1, 14, 0, 0), -0.040

In [344]:
df = data.copy()
df['Date'] = df.index

run_results = strategy_dict['HybridATRCCIStrategy']['results']
strategy_instance = run_results[0]

buys = strategy_instance.buy_signals
sells = strategy_instance.sell_signals

# upper_threshold = strategy_instance.upper_bounds
# lower_threshold = strategy_instance.lower_bounds
# print(upper_threshold)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Close'],
    mode='lines',
    name='Price'
))

fig.add_trace(go.Scatter(
    x=[signal[0] for signal in buys],
    y=[signal[1] for signal in buys],
    mode='markers',
    marker=dict(size=10, color='green', symbol='triangle-up'),
    name='Buy Signal'
))

fig.add_trace(go.Scatter(
    x=[signal[0] for signal in sells],
    y=[signal[1] for signal in sells],
    mode='markers',
    marker=dict(size=10, color='red', symbol='triangle-down'),
    name='Sell Signal'
))

# fig.add_trace(go.Scatter(
#     x=[signal[0] for signal in upper_threshold],
#     y=[signal[1] for signal in upper_threshold],
#     mode='lines',
#     name='Upper Bound'
# ))

# fig.add_trace(go.Scatter(
#     x=[signal[0] for signal in lower_threshold],
#     y=[signal[1] for signal in lower_threshold],
#     mode='lines',
#     name='Lower Bound'
# ))


fig.update_layout(
    title="Buy/Sell Signals",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_white",
    height=600,
    width=1000
)

fig.show()

In [345]:
# Create a Plotly figure
fig = go.Figure()

for strat_name, data_dict in strategy_dict.items():
    run_results = data_dict['results']
    
    # Usually returns is a list of strategy instances; we assume one strategy instance
    strategy_instance = run_results[0]
    
    # Extract the TimeReturn analyzer
    time_return_dict = strategy_instance.analyzers.timereturn.get_analysis()
    
    # Convert to a pandas Series
    returns_series = pd.Series(time_return_dict)
    
    # Calculate cumulative returns: (1 + r).cumprod() - 1
    cumulative_returns = (1 + returns_series).cumprod() - 1
    
    # Plot with Plotly
    fig.add_trace(go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.values,
        mode='lines',
        name=f"{strat_name} (Cumulative)"
    ))

# Update figure layout
fig.update_layout(
    title="Strategy Comparison: Cumulative Returns",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    template="plotly_white",
    width=1200,
    height=700
)

# Show the figure
fig.show()
